# Importando o Dataset

In [5]:
import json

In [6]:
teste = open('Dataset.txt', 'r')
dataset = json.load(teste)

# Chromadb - Armazenando o dataset

In [7]:
import sys
print(sys.executable)

C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


In [8]:

for i in dataset:
    a = f"""{{
      "date":"{i['date']}",
      "time":"{i['time']}",
      "period":"{i['period']}",
      "day_of_week":"{i['day_of_week']}",
      "season":"{i['season']}",
      "session":"{i['session']}"
      }}
      
      {i["description"]}
      """

    if (i['date'] == '2025-02-12'):
        print(a)

{
      "date":"2025-02-12",
      "time":"07:20:00",
      "period":"manhÃ£",
      "day_of_week":"quarta-feira",
      "season":"verÃ£o",
      "session":"cafÃ©-da-manhÃ£"
      }

      Preparou cafÃ© coado em filtro de papel, observando a primeira luz do dia refletir no lÃ­quido Ã¢mbar enquanto a chaleira assobiava. Um pÃ£o francÃªs amanhecido foi transformado em torradas douradas, untadas com manteiga que derretia em veios translÃºcidos. Do quintal, vinha o barulho de pÃ¡ssaros disputando migalhas caÃ­das da mesa externa, enquanto o cheiro de terra molhada da chuva noturna ainda pairava no ar.
      
{
      "date":"2025-02-12",
      "time":"12:50:00",
      "period":"meio-dia",
      "day_of_week":"quarta-feira",
      "season":"verÃ£o",
      "session":"almoÃ§o"
      }

      Reaqueceu arroz do dia anterior, adicionando ervilhas congeladas que estalavam na panela. Cortou tomates maduros em fatias desiguais, cujo suco se misturou ao azeite no prato formando poÃ§as rosadas. Dura

In [9]:
import ollama
import chromadb
import numpy as np

In [10]:
client = chromadb.PersistentClient(path="chroma/")
collection = client.get_or_create_collection(name="temporal")

ollama.pull("granite-embedding")

for d, i in enumerate(dataset):
    a = f"""{{
      "date":"{i['date']}",
      "time":"{i['time']}",
      "period":"{i['period']}",
      "day_of_week":"{i['day_of_week']}",
      "season":"{i['season']}",
      "session":"{i['session']}"
      }}
      """
    
    description = i["description"]

    embedding = np.array(ollama.embed(model="granite-embedding:278m", input=a)["embeddings"])
    description_embedding = np.array(ollama.embed(model="granite-embedding:278m", input=description)["embeddings"])

    # Reduzir peso da description em: reduzir 80%
    description_embedding *= 0.2

    # Combinar embeddings
    final_embedding = embedding + description_embedding

    collection.add(
        ids=[str(d)],
        embeddings=final_embedding.tolist(),
        documents=[f"{a}\n\n {description}"]
    )

Add of existing embedding ID: 0
Add of existing embedding ID: 1
Add of existing embedding ID: 2
Add of existing embedding ID: 3
Add of existing embedding ID: 4
Add of existing embedding ID: 5
Add of existing embedding ID: 6
Add of existing embedding ID: 7
Add of existing embedding ID: 8
Add of existing embedding ID: 9
Add of existing embedding ID: 10
Add of existing embedding ID: 11
Add of existing embedding ID: 12
Add of existing embedding ID: 13
Add of existing embedding ID: 14
Add of existing embedding ID: 15
Add of existing embedding ID: 16
Add of existing embedding ID: 17
Add of existing embedding ID: 18
Add of existing embedding ID: 19
Add of existing embedding ID: 20
Add of existing embedding ID: 21
Add of existing embedding ID: 22
Add of existing embedding ID: 23
Add of existing embedding ID: 24
Add of existing embedding ID: 25
Add of existing embedding ID: 26
Add of existing embedding ID: 27
Add of existing embedding ID: 28
Add of existing embedding ID: 29
Add of existing embe

In [11]:
collection.count()

44

# LLM - Requisição Inicial

In [12]:
pergunta = "O que almocei no dia 2025-02-26?"

In [13]:
embeddingPergunta = ollama.embed(model="granite-embedding:278m", input=pergunta)["embeddings"]
memorias = collection.query(query_embeddings=embeddingPergunta, n_results=6)["documents"]

In [14]:
memorias

[['{\n      "date":"2025-02-26",\n      "time":"20:35:00",\n      "period":"noite",\n      "day_of_week":"terÃ§a",\n      "season":"verÃ£o",\n      "session":"jantar"\n      }\n      \n\n O jantar em casa foi um momento de tranquilidade. A luz suave das velas criava um ambiente Ã\xadntimo, enquanto a mÃºsica clÃ¡ssica preenchia o ar. O sabor do frango assado com ervas pairava no ar, acompanhado por legumes frescos e crocantes. A conversa com a famÃ\xadlia era leve e descontraÃ\xadda, histÃ³rias do dia sendo compartilhadas, risadas e carinho. \n Antes de dormir, um livro nas mÃ£os, a luz suave do abajur iluminando as pÃ¡ginas. A histÃ³ria envolvente me transportava para outro mundo, um refÃºgio da realidade. A sensaÃ§Ã£o de paz e relaxamento me invadia, o corpo cansado, mas a mente tranquila. O sono chegou suavemente, um descanso merecido apÃ³s um dia cheio de experiÃªncias.',
  '{\n      "date":"2025-02-26",\n      "time":"12:20:00",\n      "period":"dia",\n      "day_of_week":"terÃ§a"

In [18]:
memorias[0]

['{\n      "date":"2025-02-26",\n      "time":"20:35:00",\n      "period":"noite",\n      "day_of_week":"terÃ§a",\n      "season":"verÃ£o",\n      "session":"jantar"\n      }\n      \n\n O jantar em casa foi um momento de tranquilidade. A luz suave das velas criava um ambiente Ã\xadntimo, enquanto a mÃºsica clÃ¡ssica preenchia o ar. O sabor do frango assado com ervas pairava no ar, acompanhado por legumes frescos e crocantes. A conversa com a famÃ\xadlia era leve e descontraÃ\xadda, histÃ³rias do dia sendo compartilhadas, risadas e carinho. \n Antes de dormir, um livro nas mÃ£os, a luz suave do abajur iluminando as pÃ¡ginas. A histÃ³ria envolvente me transportava para outro mundo, um refÃºgio da realidade. A sensaÃ§Ã£o de paz e relaxamento me invadia, o corpo cansado, mas a mente tranquila. O sono chegou suavemente, um descanso merecido apÃ³s um dia cheio de experiÃªncias.',
 '{\n      "date":"2025-02-26",\n      "time":"12:20:00",\n      "period":"dia",\n      "day_of_week":"terÃ§a",\

In [19]:
from ollama import chat
from ollama import ChatResponse

In [23]:
message =  f"""
Com base nos embeddings gerados sobre a minha pergunta:  
Embeddings : {memorias[0]}

- **Pergunta**: {pergunta}  
- **Critérios**:  
1. Priorize embeddings que tenham **similaridade semântica** com palavras-chave ou contextos da pergunta.  
2. Calcule a **similaridade de cosseno** entre a pergunta e os embeddings para ranqueamento.  
3. Gere uma **resposta direta** à pergunta, combinando informações dos embeddings mais relevantes.  
4. Retorne os **3 registros mais relevantes**, incluindo datas/horários associados a cada um.  
"""
messages = [{'role': 'user', 'content': message}]
    

response: ChatResponse = chat(model='deepseek-r1:14b', messages=messages)

print(response['message']['content'])

<think>
Ok, preciso responder à pergunta: "O que almocei no dia 2025-02-26?" Primeiro, vou olhar os embeddings fornecidos e identificar aqueles que sejam relevantes.

Vendo as datas, dois embeddings são do dia 26 de fevereiro. Um é um jantar à noite, e o outro é almoço. A palavra-chave "almocei" está diretamente associada ao embedding com a sessão "almoco". Portanto, aquele é o mais relevante.

Os outros embeddings são de datas posteriores ou se referem a jantares, que não respondem à pergunta sobre o almoço do dia 26. Assim, os três registros mais relevantes incluem o almoço do dia 26 e dois registros de jantar, mas o principal é aquele.
</think>

**Resposta:**  
No dia 2025-02-26, você almoçou em um pequeno restaurante italiano no centro da cidade. O ambiente era acolhedor, com aroma de manjericão e tomate, e você saboreou massa fresca cozida al dente acompanhada por vinho tinto encorpado. A conversa com um velho amigo foi alegre, repleta de risadas e histórias compartilhadas.

**3 R